In [1]:
import geopandas as gpd
import xarray as xr
import numpy as np
import rioxarray as rxr
import os
from math import e
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
IDir = r'/Volumes/PhD/imagery/masters/ROC/M1_ROC.shp'
#IDir = r'/Volumes/PhD/imagery/masters/output/ROC/design_storm/M1_design_storm.shp'
ODir = r'/Volumes/PhD/imagery/masters/output/ROC/'
dNBR_df = gpd.read_file(IDir)

Qs = pd.read_csv(r'/Volumes/PhD/imagery/masters/output/normality/normality_test_results.csv')
#dnbr_basins = pd.read_csv(r'/Volumes/PhD/imagery/masters/hyper/basins.csv')
#dnbr_basins
dNBR_df
#Qs

,BASIN_ID,Shape_Leng,Shape_Area,Segment_ID,Acc_Cl,Perim_Buff,InsidePeri,OutsidePer,Fire_ID,Fire_Segme,...,Field,ROC,nearest_st,distance_t,RI_index,RI,Mean_dNBR,Med_dNBR,PF_dNBR,geometry
0,17499,7120.0,1.429800e+06,17499,1.0,10,100,0,thm2017,thm2017_17499,...,1,1,23,2055.346175,23,5.08,0.471854,0.477169,0.459052,"POLYGON ((295619.84 3816329.559, 295619.84 381..."
1,3115,1120.0,4.180000e+04,3115,1.0,10,100,10,thm2017,thm2017_3115,...,4,0,29,1595.604025,29,7.11,0.072363,0.014718,-0.019005,"MULTIPOLYGON (((298029.84 3825079.559, 298029...."
2,3589,4020.0,4.313000e+05,3589,1.0,10,100,0,thm2017,thm2017_3589,...,4,0,29,1881.463927,29,7.11,0.216498,0.051373,-0.020064,"POLYGON ((297059.84 3824619.559, 297069.84 382..."
3,5683,4420.0,5.659000e+05,5683,1.0,10,101,12,thm2017,thm2017_5683,...,4,0,21,1702.372795,21,12.95,0.692429,0.702799,0.724929,"POLYGON ((283709.84 3822919.559, 283689.84 382..."
4,5920,16620.0,7.486600e+06,5920,1.0,10,100,0,thm2017,thm2017_5920,...,4,0,37,1525.053372,37,6.10,0.605956,0.651802,0.753237,"POLYGON ((303389.84 3819559.559, 303389.84 381..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
732,15065,1720.0,9.550000e+04,15065,1.0,10,100,0,thm2017,thm2017_15065,...,0,0,10,1173.707451,10,9.14,0.164376,0.111513,0.055359,"POLYGON ((289119.84 3817119.559, 289079.84 381..."
733,35852,2080.0,8.360000e+04,35852,1.0,10,101,12,thm2017,thm2017_35852,...,0,0,25,2151.287445,25,7.37,0.284851,0.279801,0.261897,"MULTIPOLYGON (((278059.84 3804989.559, 278069...."
734,36553,1460.0,4.570000e+04,36553,1.0,10,100,11,thm2017,thm2017_36553,...,0,0,25,1675.063365,25,7.37,0.348485,0.339397,0.331989,"MULTIPOLYGON (((277049.84 3804469.559, 277049...."
735,36955,2380.0,1.098000e+05,36955,1.0,10,100,11,thm2017,thm2017_36955,...,0,0,25,1352.291607,25,7.37,0.208930,0.202108,0.175856,"POLYGON ((277699.84 3804059.559, 277689.84 380..."


In [81]:
#subset qs by dNBR_df basin id
Qs_subset = Qs[Qs['Basin_ID'].isin(dNBR_df['BASIN_ID'])]
Qs_subset

,Basin_ID,SW_Statistic,SW_P_value,SW_Normal,DP_Statistic,DP_P_value,DP_Normal,Dip_Statistic,Dip_P_value,Bimodal,Q1,Q3
1,10048,0.985049,1.287702e-05,1,12.362344,2.068002e-03,1,0.013168,0.773796,0,0.316569,0.609754
2,10058,0.891842,7.096566e-06,1,10.681829,4.791486e-03,1,0.025264,0.990390,0,0.416359,0.504033
3,10060,0.912353,5.436477e-04,1,3.475222,1.759402e-01,0,0.041927,0.667167,0,0.381505,0.477545
4,10104,0.853367,1.239101e-08,1,27.076409,1.319570e-06,1,0.028374,0.848372,0,0.429042,0.529377
7,10155,0.956637,8.430536e-20,1,144.968298,3.315386e-32,1,0.007702,0.906938,0,0.400543,0.767731
...,...,...,...,...,...,...,...,...,...,...,...,...
1725,9865,0.945665,7.641928e-03,1,3.039715,2.187431e-01,0,0.060413,0.077533,2,0.408120,0.769237
1730,9906,0.984439,4.395452e-02,1,4.320410,1.153015e-01,0,0.029931,0.315710,0,0.319345,0.547760
1732,9959,0.920445,7.391726e-07,1,32.367704,9.363588e-08,1,0.044982,0.044959,1,0.658777,0.760692
1733,9974,0.958141,9.943072e-25,1,147.474170,9.470892e-33,1,0.004796,0.992495,0,0.556928,0.799709


In [82]:
# merge dNBR_df with Qs_subset on 'Basin_ID'
dNBR_df = dNBR_df.merge(Qs_subset, left_on='BASIN_ID', right_on='Basin_ID', how='left')
dNBR_df

,BASIN_ID,Shape_Leng,Shape_Area,Segment_ID,Acc_Cl,Perim_Buff,InsidePeri,OutsidePer,Fire_ID,Fire_Segme,...,SW_P_value,SW_Normal,DP_Statistic,DP_P_value,DP_Normal,Dip_Statistic,Dip_P_value,Bimodal,Q1,Q3
0,17499,7120.0,1.429800e+06,17499,1.0,10,100,0,thm2017,thm2017_17499,...,1.251742e-14,1,110.758458,8.894190e-25,1,0.003726,0.992963,0,0.391012,0.562539
1,3115,1120.0,4.180000e+04,3115,1.0,10,100,10,thm2017,thm2017_3115,...,1.406720e-07,1,16.778703,2.272747e-04,1,0.034850,0.506168,0,-0.030410,0.187332
2,3589,4020.0,4.313000e+05,3589,1.0,10,100,0,thm2017,thm2017_3589,...,7.408916e-31,1,3462.595480,0.000000e+00,1,0.049018,0.000000,1,-0.026922,0.505429
3,5683,4420.0,5.659000e+05,5683,1.0,10,101,12,thm2017,thm2017_5683,...,1.709398e-13,1,90.400038,2.343588e-20,1,0.005896,0.992896,0,0.624122,0.776981
4,5920,16620.0,7.486600e+06,5920,1.0,10,100,0,thm2017,thm2017_5920,...,1.530483e-63,1,1946.688408,0.000000e+00,1,0.001218,1.000000,0,0.488693,0.767520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
732,15065,1720.0,9.550000e+04,15065,1.0,10,100,0,thm2017,thm2017_15065,...,7.848881e-11,1,26.800375,1.514860e-06,1,0.023733,0.475326,0,0.029212,0.301205
733,35852,2080.0,8.360000e+04,35852,1.0,10,101,12,thm2017,thm2017_35852,...,7.392152e-05,1,21.764447,1.878929e-05,1,0.030156,0.213216,0,0.239544,0.324979
734,36553,1460.0,4.570000e+04,36553,1.0,10,100,11,thm2017,thm2017_36553,...,1.097923e-03,1,17.217529,1.824992e-04,1,0.021545,0.988148,0,0.303964,0.384710
735,36955,2380.0,1.098000e+05,36955,1.0,10,100,11,thm2017,thm2017_36955,...,1.111881e-03,1,24.634988,4.472809e-06,1,0.013880,0.990638,0,0.128743,0.295251


In [3]:
def M1(dnbr, Value, RI):
    
    # Coefficients for Southern California
    b = -3.63
    b_1 = 0.41
    b_2 = 0.67
    b_3 = 0.7
    
    x_1 = dnbr['L_X1']
    x_3 = dnbr['L_X3']
    x_2 = dnbr[f'{Value}']
    lnx =  b + (b_1 * x_1 * RI) + (b_2 * x_2 * RI) + (b_3 * x_3 * RI)
    prob = (e ** lnx) / (1.0 + e ** lnx)
    return {
        'BASIN_ID': basin_id,
        'RI': RI *4,
        'M1P': prob
    }
    

In [4]:
def HM1(dnbr, Value, RI):
    
    # Coefficients for Southern California
    b = -3.63
    b_1 = 0.41
    b_2 = 0.67
    b_3 = 0.7
    
    x_1 = dnbr['X1']
    x_3 = dnbr['X3']
    x_2 = dnbr[f'{Value}']
    lnx =  b + (b_1 * x_1 * RI) + (b_2 * x_2 * RI) + (b_3 * x_3 * RI)
    prob = (e ** lnx) / (1.0 + e ** lnx)
    return {
        'BASIN_ID': basin_id,
        'RI': RI *4,
        'M1P': prob
    }
    

In [7]:
results = []
results_1 = []
results_2 = []
for basin_id in dnbr_basins['BASIN_ID'].unique():
    basin_data = dnbr_basins[dnbr_basins['BASIN_ID'] == basin_id].iloc[0]
    R = [3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]
    for RI in R:
        result = HM1(basin_data, 'Mean_dNBR', RI)
        results.append(result)
        M1P_Median = HM1(basin_data, 'Med_dNBR', RI)
        results_1.append(M1P_Median)
        M1P_PF = HM1(basin_data, 'PF_dNBR', RI)
        results_2.append(M1P_PF)


final_df = pd.DataFrame(results)
final_df_1 = pd.DataFrame(results_1)
final_df_2 = pd.DataFrame(results_2)


In [8]:
final_df.to_csv(os.path.join(ODir, 'HM1P_mean.csv'))
final_df_1.to_csv(os.path.join(ODir,'HM1P_med.csv'))
final_df_2.to_csv(os.path.join(ODir, 'HM1P_PF.csv'))

In [11]:
All_df = pd.DataFrame()
test = pd.DataFrame()

R = [3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]
for RI in R:
    M1P_Mean = M1(dNBR_df, 'Mean_dNBR')
    M1P_Median = M1(dNBR_df, 'Med_dNBR')
    M1P_PF = M1(dNBR_df, 'PF_dNBR')
    R_val = int(RI*4)
    testdf = {
        'RI': R_val,
        'M1P_M': M1P_Mean
    }
    testdf = pd.DataFrame(testdf)
    test = pd.concat([test, testdf], axis=1)
    M1P_df = {
        f'M1P_Mn_{R_val}': M1P_Mean,
        f'M1P_Md_{R_val}': M1P_Median,
        f'M1P_PF_{R_val}': M1P_PF
        }
    M1P_df = pd.DataFrame(M1P_df)   
    All_df = pd.concat([All_df, M1P_df], axis=1)

All_df




,M1P_Mn_12,M1P_Md_12,M1P_PF_12,M1P_Mn_16,M1P_Md_16,M1P_PF_16,M1P_Mn_20,M1P_Md_20,M1P_PF_20,M1P_Mn_24,...,M1P_PF_28,M1P_Mn_32,M1P_Md_32,M1P_PF_32,M1P_Mn_36,M1P_Md_36,M1P_PF_36,M1P_Mn_40,M1P_Md_40,M1P_PF_40
0,0.064164,0.056530,0.053738,0.086012,0.072894,0.068209,0.114390,0.093525,0.086222,0.150589,...,0.135529,0.250367,0.189059,0.168113,0.314323,0.234262,0.206657,0.386199,0.286454,0.251368
1,0.102490,0.084037,0.071389,0.156679,0.121858,0.098791,0.232110,0.173476,0.135180,0.329661,...,0.241169,0.565548,0.420700,0.311854,0.679271,0.523448,0.392540,0.775065,0.624249,0.479554
2,0.117073,0.107684,0.084992,0.184835,0.166659,0.123629,0.279405,0.248923,0.176443,0.398698,...,0.330730,0.659741,0.601332,0.428735,0.768286,0.714256,0.532669,0.850073,0.805538,0.633842
3,0.115671,0.102787,0.075314,0.182111,0.157248,0.105863,0.274854,0.233069,0.146836,0.392179,...,0.266688,0.651533,0.567658,0.345830,0.760925,0.681373,0.434541,0.844189,0.776932,0.527653
4,0.201483,0.209799,0.215912,0.348403,0.363968,0.375303,0.531189,0.552251,0.567236,0.705973,...,0.861859,0.915124,0.925092,0.931562,0.958067,0.963792,0.967424,0.979764,0.982868,0.984800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1731,0.060798,0.056792,0.052333,0.080175,0.073337,0.065876,0.105037,0.094220,0.082618,0.136466,...,0.128061,0.222709,0.191072,0.157935,0.278394,0.236910,0.193234,0.341878,0.289806,0.234228
1732,0.067893,0.067951,0.069137,0.092567,0.092670,0.094773,0.125006,0.125173,0.128602,0.166723,...,0.226756,0.281836,0.282330,0.292475,0.354677,0.355305,0.368171,0.434942,0.435691,0.450972
1733,0.081784,0.080229,0.077522,0.117692,0.114831,0.109877,0.166508,0.161732,0.153489,0.230285,...,0.281205,0.401568,0.388260,0.364939,0.501237,0.485577,0.457729,0.600808,0.583999,0.553547
1734,0.225089,0.219513,0.199548,0.392137,0.381935,0.344757,0.588938,0.575865,0.526175,0.760877,...,0.831838,0.940101,0.935070,0.912589,0.972111,0.969364,0.956588,0.987247,0.985820,0.978950


In [12]:


merged_gdf = gpd.GeoDataFrame(
    pd.merge(dNBR_df, All_df, left_index=True, right_index=True, how="left")
)

In [13]:
merged_gdf

,BASIN_ID,Shape_Leng,Shape_Area,Segment_ID,Acc_Cl,Perim_Buff,InsidePeri,OutsidePer,Fire_ID,Fire_Segme,...,M1P_PF_28,M1P_Mn_32,M1P_Md_32,M1P_PF_32,M1P_Mn_36,M1P_Md_36,M1P_PF_36,M1P_Mn_40,M1P_Md_40,M1P_PF_40
0,123,16060.0,6.322700e+06,123,1.0,10,0,0,thm2017,thm2017_123,...,0.135529,0.250367,0.189059,0.168113,0.314323,0.234262,0.206657,0.386199,0.286454,0.251368
1,155,3140.0,2.716000e+05,155,1.0,10,101,12,thm2017,thm2017_155,...,0.241169,0.565548,0.420700,0.311854,0.679271,0.523448,0.392540,0.775065,0.624249,0.479554
2,167,1800.0,7.260000e+04,167,1.0,10,100,11,thm2017,thm2017_167,...,0.330730,0.659741,0.601332,0.428735,0.768286,0.714256,0.532669,0.850073,0.805538,0.633842
3,172,2720.0,2.340000e+05,172,1.0,10,101,12,thm2017,thm2017_172,...,0.266688,0.651533,0.567658,0.345830,0.760925,0.681373,0.434541,0.844189,0.776932,0.527653
4,184,1540.0,4.900000e+04,184,1.0,10,100,11,thm2017,thm2017_184,...,0.861859,0.915124,0.925092,0.931562,0.958067,0.963792,0.967424,0.979764,0.982868,0.984800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1731,44218,2880.0,2.577000e+05,44218,1.0,10,100,11,thm2017,thm2017_44218,...,0.128061,0.222709,0.191072,0.157935,0.278394,0.236910,0.193234,0.341878,0.289806,0.234228
1732,44238,6260.0,8.933000e+05,44238,1.0,10,100,0,thm2017,thm2017_44238,...,0.226756,0.281836,0.282330,0.292475,0.354677,0.355305,0.368171,0.434942,0.435691,0.450972
1733,44256,1140.0,3.360000e+04,44256,1.0,10,100,11,thm2017,thm2017_44256,...,0.281205,0.401568,0.388260,0.364939,0.501237,0.485577,0.457729,0.600808,0.583999,0.553547
1734,44320,3020.0,1.722000e+05,44320,1.0,10,0,0,thm2017,thm2017_44320,...,0.831838,0.940101,0.935070,0.912589,0.972111,0.969364,0.956588,0.987247,0.985820,0.978950


In [15]:
# Step 1: Verify RI-related columns
ri_columns = [col for col in merged_gdf.columns if "MIP" in col]  # Adjust keyword as needed
print("RI Columns:", ri_columns)

# Step 2: Melt the DataFrame
long_df = pd.melt(
    merged_gdf,
    id_vars=["BASIN_ID"],  # Ensure BASIN_ID exists in the DataFrame
    value_vars=ri_columns,  # Use identified RI-related columns
    var_name="Variable",
    value_name="Value"
)
print("Melted DataFrame:")
print(long_df.head())

# Step 3: Extract RI values and variable types
long_df["RI"] = long_df["Variable"].str.extract(r"(\d+)$").astype(float)
long_df["Variable"] = long_df["Variable"].str.extract(r"^(.*)_R")

print("After Extracting RI and Variables:")
print(long_df.head())

# Step 4: Optional - Pivot the DataFrame
reshaped_df = long_df.pivot_table(
    index=["BASIN_ID", "RI"],  # Use BASIN_ID and RI as unique keys
    columns="Variable",  # Spread variables into separate columns
    values="Value"
).reset_index()

print("Reshaped DataFrame:")

RI Columns: []
Melted DataFrame:
Empty DataFrame
Columns: [BASIN_ID, Variable, Value]
Index: []
After Extracting RI and Variables:
Empty DataFrame
Columns: [BASIN_ID, Variable, Value, RI]
Index: []
Reshaped DataFrame:


In [28]:
merged_gdf.to_file(os.path.join(ODir, 'M1P_design_storm.shp'))

In [83]:
Basin = []

def M1ROC(dnbr, Value):
    # Coefficients for Southern California
    b = -3.63
    b_1 = 0.41
    b_2 = 0.67
    b_3 = 0.7
    

    Basin.append(dnbr['BASIN_ID'])
    x_1 = dnbr['L_X1']
    x_3 = dnbr['L_X3']
    x_2 = dnbr[f'{Value}']
    RI = dnbr['RI']

    lnx =  b + (b_1 * x_1 * RI) + (b_2 * x_2 * RI) + (b_3 * x_3 * RI)
    prob = (e ** lnx) / (1.0 + e ** lnx)
    return prob
    

In [84]:

M1_Q1ROC = M1ROC(dNBR_df, 'Q1')
#M1_ROC = M1_ROC.to_DataFrame()


In [85]:
M1_Q3ROC = M1ROC(dNBR_df, 'Q3')
M1_Q3ROC


0      0.526585
1      0.231178
2      0.630905
3      0.999606
4      0.915496
         ...   
732    0.513164
733    0.522545
734    0.512990
735    0.375204
736    0.504082
Length: 737, dtype: float64

In [86]:
QM1_df = {
    'Q1p': M1_Q1ROC,
    'Q3P': M1_Q3ROC
}
QM1_df

{'Q1p': 0      0.382874
 1      0.096308
 2      0.119217
 3      0.998519
 4      0.776103
          ...   
 732    0.166178
 733    0.417845
 734    0.414172
 735    0.208803
 736    0.266099
 Length: 737, dtype: float64,
 'Q3P': 0      0.526585
 1      0.231178
 2      0.630905
 3      0.999606
 4      0.915496
          ...   
 732    0.513164
 733    0.522545
 734    0.512990
 735    0.375204
 736    0.504082
 Length: 737, dtype: float64}

In [87]:
QM1_df = pd.DataFrame(QM1_df)
QM1_df

,Q1p,Q3P
0,0.382874,0.526585
1,0.096308,0.231178
2,0.119217,0.630905
3,0.998519,0.999606
4,0.776103,0.915496
...,...,...
732,0.166178,0.513164
733,0.417845,0.522545
734,0.414172,0.512990
735,0.208803,0.375204


In [91]:
merged_gdf = gpd.GeoDataFrame(
    pd.merge(dNBR_df, QM1_df, left_index=True, right_index=True, how="left")
)
merged_gdf.head()
#print all columns in merged_gdf
print("Merged GeoDataFrame Columns:")
print(merged_gdf.columns)

Merged GeoDataFrame Columns:
Index(['BASIN_ID', 'Shape_Leng', 'Shape_Area', 'Segment_ID', 'Acc_Cl',
       'Perim_Buff', 'InsidePeri', 'OutsidePer', 'Fire_ID', 'Fire_Segme',
       'L_X1', 'L_X2', 'L_X3', 'V_X1', 'V_X2', 'Shape_Le_1', 'R', 'V_X3', 'X',
       'ExpX', 'P', 'PCl', 'PCl_Legend', 'LnV', 'Volume', 'VolMin', 'VolMax',
       'VolCl', 'VolCl_Lege', 'CombHaz', 'CombHazCl', 'CombHazCl_', 'area',
       'Field', 'ROC', 'nearest_st', 'distance_t', 'RI_index', 'RI',
       'Mean_dNBR', 'Med_dNBR', 'PF_dNBR', 'geometry', 'Basin_ID',
       'SW_Statistic', 'SW_P_value', 'SW_Normal', 'DP_Statistic', 'DP_P_value',
       'DP_Normal', 'Dip_Statistic', 'Dip_P_value', 'Bimodal', 'Q1', 'Q3',
       'Q1p', 'Q3P'],
      dtype='object')


In [92]:
merged_gdf.to_file(os.path.join(ODir, 'QM1P_ROC.shp'))

/var/folders/mm/sjj0b1k91z39pkxfzn0rj_1h0000gn/T/ipykernel_62432/3999804140.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  merged_gdf.to_file(os.path.join(ODir, 'QM1P_ROC.shp'))
/Users/samueltaylor/Documents/GitHub/PHA_burn_severity/.venv/lib/python3.9/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Basin_ID' to 'Basin_ID_1'
  ogr_write(
/Users/samueltaylor/Documents/GitHub/PHA_burn_severity/.venv/lib/python3.9/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'SW_Statistic' to 'SW_Statist'
  ogr_write(
/Users/samueltaylor/Documents/GitHub/PHA_burn_severity/.venv/lib/python3.9/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'DP_Statistic' to 'DP_Statist'
  ogr_write(
/Users/samueltaylor/Documents/GitHub/PHA_burn_severity/.venv/lib/python3.9/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: '